In [1]:
import pandas as pd 

In [2]:
from aind_data_access_api.document_db import MetadataDbClient

API_GATEWAY_HOST = "api.allenneuraldynamics.org"
DATABASE = 'metadata_index'
COLLECTION = 'data_assets'

docdb_api_client = MetadataDbClient(
   host=API_GATEWAY_HOST,
   database=DATABASE,
   collection=COLLECTION,
)
print(docdb_api_client._base_url)

https://api.allenneuraldynamics.org/v1/metadata_index/data_assets


In [3]:
aggregate = [
  {
    "$match": {
      "name": {
        "$regex": "single-plane-ophys",  # add .*processed or .*raw to narrow search 
        "$options": "i"
      },
      "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim"
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] }
    }
  },
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [4]:
aggregate = [
  {
    "$match": {
      "name": {
        "$regex": "single-plane-ophys",
        "$options": "i"
      },
      "data_description.project_name": "Brain Computer Interface",
      "session.session_type": "BCI single neuron stim",
      "name": {
        "$not": {
          "$regex": "processed",
          "$options": "i"
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": "$data_description.subject_id",
      "session_type": "$session.session_type", 
      "genotype": "$subject.genotype", 
      "virus_injection_materials": "$procedures.subject_procedures.procedures.injection_materials.name",
      "session_number": {
        "$filter": {
          "input": "$session.stimulus_epochs",
          "as": "epoch",
          "cond": { "$eq": ["$$epoch.stimulus_name", "single neuron BCI conditioning"] }
        }
      }
    }
  },
  {
    "$project": {
      "name": 1,
      "subject_id": 1,
      "session_type": 1,
      "genotype": 1,
      "virus_injection_materials": 1,
      "session_number": { "$arrayElemAt": ["$session_number.session_number", 0] }
    }
  },
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}, 
  {'$unwind': {'path': '$virus_injection_materials', 'preserveNullAndEmptyArrays': True}}
]

records = docdb_api_client.aggregate_docdb_records(
    pipeline = aggregate,
)

In [5]:
name = [] 
subject_id = [] 
session_type = [] 
genotype = []
virus = []
session_number = []

for record in records: 
    name.append(record["name"])
    subject_id.append(record["subject_id"])
    session_type.append(record["session_type"]) 
    genotype.append(record["genotype"])
    virus.append(record["virus_injection_materials"][0])
    session_number.append(record["session_number"])
    
    
bci_df = pd.DataFrame() 
bci_df["name"] = name 
bci_df["subject_id"] = subject_id
bci_df["session_type"] = session_type
bci_df["genotype"] = genotype
bci_df["virus"] = virus
bci_df["session_number"] = session_number

In [7]:
bci_df

,name,subject_id,session_type,genotype,virus,session_number
0,single-plane-ophys_772414_2025-01-28_16-19-17,772414,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,6.0
1,single-plane-ophys_772414_2025-01-23_16-03-51,772414,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,3.0
2,single-plane-ophys_731015_2025-01-28_18-56-35,731015,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,22.0
3,single-plane-ophys_754303_2025-01-23_19-48-32,754303,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,3.0
4,single-plane-ophys_766719_2025-01-23_18-24-57,766719,BCI single neuron stim,Camk2a-tTA/wt;TetO-jGCaMP8s-01/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,3.0
5,single-plane-ophys_731015_2025-01-10_18-06-31,731015,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,18.0
6,single-plane-ophys_740369_2025-01-10_20-21-40,740369,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,23.0
7,single-plane-ophys_740369_2025-02-03_19-18-31,740369,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,27.0
8,single-plane-ophys_772414_2025-02-03_12-56-57,772414,BCI single neuron stim,Camk2a-tTA/wt;tetO-GCaMP6s/wt,pAAV-CaMKIIa-ChRmine-oScarlet-Kv2.1-WPRE - 7413,10.0
9,single-plane-ophys_767715_2025-02-03_17-30-05,767715,BCI single neuron stim,Slc17a6-IRES-Cre/wt;Ai230(TIT2L-XCaMPG-WPRE-IC...,pAAV-hSyn1-RiboL1-GCaMP8s-WPRE,3.0


In [9]:
# bci_df.to_csv('/data/metadata.csv', index= False) 